In [0]:
%sql
-- Step 1: Close old records if there are changes
MERGE INTO product_scd2 AS target
USING product_source AS source
ON target.product_id = source.product_id AND target.is_current = 1
WHEN MATCHED AND (
    target.product_name != source.product_name OR
    target.category != source.category OR
    target.price != source.price OR
    target.product_description != source.product_description
)
THEN 
UPDATE SET 
    target.effective_end_date = CURRENT_DATE, 
    target.is_current = 0;

-- Step 2: Insert new versions for only updated records (Prevent Duplicates)
INSERT INTO product_scd2 (
    product_id, product_name, category, price, product_description, 
    effective_start_date, effective_end_date, is_current
)
SELECT 
    source.product_id, source.product_name, source.category, 
    source.price, source.product_description, CURRENT_DATE, NULL, 1
FROM product_source AS source
WHERE EXISTS (
    SELECT 1 FROM product_scd2 AS target
    WHERE source.product_id = target.product_id 
    AND target.is_current = 0 -- Only insert if there was a closed record
)
AND NOT EXISTS (
    -- New Condition: Ensure we do NOT insert if an identical active record already exists
    SELECT 1 FROM product_scd2 AS active
    WHERE source.product_id = active.product_id 
    AND active.is_current = 1 -- Only check active records
    AND source.product_name = active.product_name
    AND source.category = active.category
    AND source.price = active.price
    AND source.product_description = active.product_description
);

--Step 3: Insert completely new records
INSERT INTO product_scd2 (
    product_id, product_name, category, price, product_description, 
    effective_start_date, effective_end_date, is_current
)
SELECT DISTINCT
    source.product_id, source.product_name, source.category, 
    source.price, source.product_description, CURRENT_DATE, NULL, 1
FROM product_source AS source
LEFT JOIN product_scd2 AS target
ON source.product_id = target.product_id
WHERE target.product_id IS NULL; -- Only insert if product_id does not exist in target
